In [3]:
import pandas as pd
import numpy as np

In [4]:
kmhas = 'kmhas/'
mydata = 'mydata/'
train_path = kmhas+'kmhas_train.txt'
test_path = kmhas+'kmhas_test.txt'
valid_path = kmhas+'kmhas_valid.txt'

In [5]:
def preprocess_data(df, Str):
    # 데이터프레임 복사본 생성
    df_copy = df.copy()

    # label 컬럼을 쉼표로 분리하여 리스트로 변환
    df_copy['label'] = df_copy['label'].apply(lambda x: [int(i) for i in x.split(',')])

    # 새로운 조건에 따라 label 컬럼 값을 재설정
    df_copy['label'] = df_copy['label'].apply(lambda labels: 0 if 3 in labels else (1 if any(label in labels for label in [0, 1, 2, 4, 5, 6, 7]) else 2))

    file_path = mydata+'mydata_'+Str+'.csv'
    with open(file_path, mode='w', encoding='utf-8-sig') as file:
        df_copy.to_csv(file, index=False)


In [6]:
# 데이터 불러오기
train_data = pd.read_csv(train_path, sep='\t')
test_data = pd.read_csv(test_path, sep='\t')
valid_data = pd.read_csv(valid_path, sep='\t')

# 컬럼 이름 설정 (예시)
train_data.columns = ['text', 'label']
test_data.columns = ['text', 'label']
valid_data.columns = ['text', 'label']

preprocess_data(train_data,'train')
preprocess_data(test_data,'test')
preprocess_data(valid_data,'valid')


In [7]:
train_mydata = pd.read_csv(mydata+'mydata_train.csv', sep=',')
test_mydata = pd.read_csv(mydata+'mydata_test.csv', sep=',')
valid_mydata = pd.read_csv(mydata+'mydata_valid.csv', sep=',')

In [8]:
train_mydata

,text,label
0,자한당틀딱들.. 악플질 고만해라.,1
1,정치적으로 편향된 평론한은 분은 별로...,2
2,적당히좀 쳐먹지.그랬냐??? 안그래도 문재인 때문에 나라 엉망진창인데...,1
3,안서는 아재들 풀발기 ㅋㄲㅋ,1
4,우와 ㅋ 능력자,2
...,...,...
78972,녹취 했다는 자체가 곽도원도 뭔 가 석연치 않은 분위기내..멀쩡하게 죄없는 사람한테...,2
78973,빨갱이당이나 쪽발이당이나ㅋ 같은 ㅆㄹ기들 ㅋ,1
78974,성상품화 aut,2
78975,열한살인뎁 섹스파티가 뭔가연?,2


In [9]:
combined_data = pd.concat([train_data, test_data, valid_data], ignore_index=True)


In [13]:
# 라벨을 분류하는 함수
def classify_label(labels):
    # 문자열 라벨을 정수 세트로 변환
    labels_set = set(int(label) for label in labels.split(','))
    if 3 in labels_set:
        return "욕설/비하/모욕"
    elif labels_set & {0, 1, 2, 4, 5, 6, 7}:
        return "성희롱/혐오"
    else:
        return "중립"


def format_data(data):
    # 빈 리스트 생성
    formatted_data = []

    
    start = 0
    while start < len(data):
        # 무작위로 1~10 사이의 수 선택
        chunk_size = np.random.randint(1, 11)
        end = start + chunk_size

        # 사용된 최대 유저 번호 초기화
        max_user_number = -1
        
        # 데이터 슬라이스
        chunk = data.iloc[start:end]
        formatted_row = []
        
        for idx, row in chunk.iterrows():
            # 사용 가능한 유저 번호 리스트 생성
            if max_user_number == -1:
                # 첫 번째 유저
                current_user_number = 0
            else:
                # 이전 유저들 중에서 무작위 선택하거나 새 유저 번호 사용
                current_user_number = np.random.randint(0, max_user_number + 2)
            
            # 최대 유저 번호 업데이트
            max_user_number = max(max_user_number, current_user_number)
            
            # 텍스트 포맷에 맞게 결합
            formatted_row.append(f"유저{current_user_number}: {row['text']}")
         # 라벨 분류
        labels_list = [row['label'] for _, row in chunk.iterrows()]
        classifications = [classify_label(label) for label in labels_list]
        
        # 라벨 정보를 문자열로 합치기
        labels_row = ','.join(classifications)
        
        # 생성된 문자열을 리스트에 추가
        formatted_data.append({'Text': ',\n'.join(formatted_row), 'Completion': labels_row})
        
        # 다음 묶음의 시작 지점 업데이트
        start = end

    # 리스트를 데이터프레임으로 변환
    return pd.DataFrame(formatted_data)

# 함수를 각 데이터 세트에 적용
valid_formatted = format_data(valid_data)
train_formatted = format_data(train_data)
test_formatted = format_data(test_data)

# 결과 확인
print(train_formatted.head())

0
8
12
15
19
20
25
31
41
50
55
61
63
66
69
75
79
89
97
105
106
115
118
119
125
126
129
135
137
142
145
152
154
159
169
179
183
190
196
202
212
215
222
232
239
241
247
251
253
261
265
267
276
283
290
293
297
305
308
316
321
325
335
344
346
350
352
357
361
363
367
376
386
390
393
402
407
408
412
417
423
433
442
450
459
467
471
477
479
487
492
498
507
511
518
526
532
536
544
551
560
570
572
573
583
584
586
591
594
596
599
600
602
608
610
619
625
628
635
642
652
653
654
658
668
672
682
686
691
701
703
709
711
713
717
722
728
735
737
746
755
757
758
759
763
767
776
784
794
804
809
818
824
825
835
845
852
861
869
876
886
887
888
889
895
896
898
900
901
907
911
914
915
921
930
933
942
945
952
960
961
967
972
976
982
989
996
1003
1006
1014
1022
1026
1029
1030
1031
1038
1041
1045
1048
1056
1065
1071
1077
1083
1090
1093
1094
1102
1110
1117
1126
1133
1135
1140
1150
1157
1159
1162
1164
1174
1180
1185
1190
1198
1200
1208
1218
1223
1227
1229
1235
1240
1243
1253
1263
1264
1274
1277
1278
1283
1290
129

40307
40315
40320
40328
40334
40344
40353
40356
40360
40370
40371
40372
40378
40382
40388
40389
40393
40396
40399
40408
40411
40416
40417
40423
40427
40434
40437
40447
40448
40449
40453
40460
40467
40475
40478
40483
40484
40494
40495
40500
40504
40505
40507
40517
40526
40531
40537
40547
40554
40560
40568
40573
40583
40593
40595
40601
40602
40603
40607
40610
40620
40630
40640
40641
40645
40652
40662
40668
40675
40676
40686
40688
40689
40696
40705
40714
40719
40722
40728
40735
40742
40744
40753
40755
40758
40763
40767
40774
40775
40781
40789
40799
40806
40813
40821
40830
40833
40839
40849
40852
40860
40865
40875
40882
40884
40887
40891
40900
40909
40914
40915
40918
40925
40928
40934
40940
40948
40953
40962
40971
40978
40983
40992
40994
41004
41011
41021
41026
41027
41028
41030
41033
41043
41052
41060
41062
41064
41073
41080
41083
41086
41089
41094
41100
41108
41111
41120
41127
41131
41140
41150
41157
41162
41169
41172
41181
41188
41194
41203
41208
41211
41212
41214
41220
41224
41226
4123

61930
61931
61937
61944
61947
61957
61961
61966
61967
61976
61986
61991
61998
62001
62011
62015
62024
62032
62033
62034
62042
62048
62050
62057
62062
62068
62077
62086
62090
62096
62101
62105
62109
62118
62122
62132
62135
62137
62145
62149
62159
62162
62163
62165
62171
62177
62184
62192
62194
62202
62211
62218
62226
62228
62238
62244
62252
62260
62261
62262
62265
62266
62272
62278
62284
62290
62297
62303
62313
62315
62320
62329
62335
62344
62345
62350
62357
62364
62366
62375
62382
62386
62395
62403
62411
62414
62424
62427
62435
62445
62453
62459
62460
62468
62473
62476
62482
62486
62491
62495
62496
62503
62510
62517
62521
62522
62523
62528
62538
62540
62544
62551
62557
62559
62569
62578
62580
62582
62590
62597
62598
62606
62615
62622
62627
62630
62634
62644
62651
62660
62667
62677
62681
62683
62691
62695
62703
62713
62715
62718
62724
62733
62738
62742
62746
62756
62761
62767
62772
62774
62781
62782
62788
62796
62798
62807
62813
62823
62826
62836
62841
62846
62853
62862
62867
62873
6287

0
5
6
13
15
16
26
27
35
41
44
52
58
60
68
78
84
86
87
94
97
107
110
114
120
124
132
141
148
154
162
163
171
173
177
183
190
199
203
206
209
210
213
217
224
229
235
239
248
249
250
251
261
262
268
275
283
287
289
295
302
303
304
309
311
318
319
329
339
340
341
346
350
351
360
370
373
374
375
383
389
391
399
408
411
420
429
434
441
451
457
460
466
471
476
478
484
490
498
507
509
516
522
529
538
543
544
545
553
557
561
568
572
577
586
595
602
607
617
621
626
633
641
645
648
655
656
659
665
675
680
687
694
699
708
715
720
728
732
735
740
742
748
755
761
765
775
779
784
785
791
796
802
809
812
814
823
826
834
841
848
858
862
864
867
872
881
884
891
895
900
902
903
904
907
915
924
934
936
942
947
955
965
967
977
985
994
995
998
1002
1003
1007
1017
1022
1029
1031
1035
1038
1039
1044
1048
1056
1060
1064
1072
1082
1087
1094
1098
1104
1106
1109
1111
1120
1123
1133
1141
1146
1151
1160
1165
1170
1172
1173
1182
1183
1193
1195
1205
1210
1213
1219
1220
1223
1227
1229
1234
1236
1242
1247
1256
1257
125

                                       combined_text  \
0  유저0: 자한당틀딱들.. 악플질 고만해라.,\n유저1: 정치적으로 편향된 평론한은 ...   
1  유저0: 공통점은 한국남자?,\n유저1: 이재진은 그게문제가아니자나 지금 ㅋㅋㅋㅋ,...   
2  유저0: 기사에 낚이는 ㅂㅅ같은 댓글러들,\n유저1: 김태리.. 나 3대째 돼지농장...   
3  유저0: 송중기는 사진에도 없구만..ㅋㅋ송혜교♡고양이.ㅋㅋㅋ,\n유저1: 도대체 ㅋ...   
4  유저0: 목사 좋아하네~~ 종교팔이 하지마라 ! 개목사 주제에..,\n유저1: 이여...   

                                   labels  C_ID  T_ID  
0  혐오, 중립, 혐오, 혐오, 중립, 중립, 중립, 중립, 욕설, 욕설     0     0  
1  중립, 중립, 욕설, 중립, 중립, 중립, 욕설, 혐오, 중립, 중립     1     0  
2  욕설, 중립, 욕설, 중립, 중립, 욕설, 욕설, 욕설, 중립, 혐오     2     0  
3  중립, 혐오, 혐오, 욕설, 혐오, 중립, 혐오, 혐오, 욕설, 혐오     3     0  
4  혐오, 중립, 혐오, 중립, 중립, 중립, 혐오, 중립, 중립, 욕설     4     0  


In [14]:
train_formatted

,Text,Completion
0,"유저0: 자한당틀딱들.. 악플질 고만해라.,\n유저0: 정치적으로 편향된 평론한은 ...","성희롱/혐오,중립,성희롱/혐오,성희롱/혐오,중립,중립,중립"
1,"유저0: 이게주간아이돌이랑머가달라...,\n유저0: 아오 슈박 회사생활도 졑깥고 돈...","중립,욕설/비하/모욕,욕설/비하/모욕,중립"
2,"유저0: 이재진은 그게문제가아니자나 지금 ㅋㅋㅋㅋ,\n유저0: 야이 OOO 색햐. ...","중립,욕설/비하/모욕,중립,중립,중립,욕설/비하/모욕,성희롱/혐오"
3,유저0: 알바풀었나 많이본뉴스 이프로그램기사 댓글하고 반응이 180도 다르네 ㅋㅋㅋ...,"중립,중립,욕설/비하/모욕,중립,욕설/비하/모욕,중립,중립"
4,유저0: 찌질한 새끼들.... 왜? 동양 남자는 무섭냐?,욕설/비하/모욕
...,...,...
14351,"유저0: 할렐루야 감사합니다, 주님,\n유저1: 이섹기 면상 보기도싫다 거짓말쟁이,...","중립,욕설/비하/모욕,중립"
14352,"유저0: 저런 사이비 종교집단에 빠진 사람들아 제발 정신차려라!!!!,\n유저1: ...","중립,욕설/비하/모욕,중립,중립,성희롱/혐오"
14353,"유저0: 좌파의 끝은 뻔하다,\n유저1: 그나이에 모아둔돈 5천이상도 없음?,\n유...","성희롱/혐오,성희롱/혐오,성희롱/혐오"
14354,"유저0: 개박살나고 패싱이나 당하는 국제왕따 문씨조선?ㅋㅋ,\n유저0: 약시 게이년...","성희롱/혐오,욕설/비하/모욕,중립,중립,성희롱/혐오,중립"


In [15]:
# DataFrame을 BOM UTF-8 인코딩으로 CSV 파일로 저장하는 함수
def save_with_bom_utf8(df, file_path):
    # DataFrame을 UTF-8 인코딩으로 문자열로 변환
    csv_string = df.to_csv(index=False, encoding='utf-8-sig')

    # 파일로 저장
    with open(file_path, 'w', encoding='utf-8-sig') as file:
        file.write(csv_string)

In [16]:
save_with_bom_utf8(train_formatted, mydata+'train_formatted.csv')

In [25]:
# CSV 파일 불러오기
df1 = pd.read_csv(mydata + 'data.csv')
df2 = pd.read_csv(mydata + 'train_formatted.csv')

# df2에서 각 레코드를 1/10 확률로 선택
mask = np.random.choice([True, False], size=len(df2), p=[1/10, 9/10])
filtered_df2 = df2[mask]

# 선택된 레코드와 df1을 합치기
combined_df = pd.concat([df1, filtered_df2], ignore_index=True)

# 데이터를 랜덤하게 섞기
shuffled_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Prefix 문자열 정의
prefix_str = ('이것은 각 유저가 채팅을 입력한 맥락이 악성인지 판단하는 분류기이다.\n'
              + '유저의 각 채팅은 맥락에 따라 분류된다.\n'
              + '분류는 (욕설/모욕/위협, 성희롱/혐오, 도배/홍보, 중립) 이다.\n'
              + '각 분류에 대한 우선순위는 (욕설/모욕/위협 > 성희롱/혐오 > 도배/스팸 > 중립) 으로 중립이 가장 낮다.\n'
              + '분류기에 의하여 각 채팅에 대한 분류를 출력한다.\n')

# 'text' 열에 Prefix 추가
shuffled_df['Text'] = shuffled_df['Text'].apply(lambda x: prefix_str + x)

# 'C_ID' 열을 추가하고, 0부터 데이터프레임 길이만큼 숫자 배열을 생성
shuffled_df['C_ID'] = np.arange(len(shuffled_df))
shuffled_df['T_ID'] = 0
# 'C_ID' 열을 데이터프레임의 맨 앞으로 이동
cols = ['C_ID'] + ['T_ID'] + [col for col in shuffled_df.columns if col != 'C_ID' and col != 'T_ID']
shuffled_df = shuffled_df[cols]


print(shuffled_df)
# 섞인 데이터를 새로운 CSV 파일로 저장
shuffled_df.to_csv(mydata+'shuffled_data.csv', index=False, encoding='utf-8-sig')

      C_ID  T_ID                                               Text  \
0        0     0  이것은 각 유저가 채팅을 입력한 맥락이 악성인지 판단하는 분류기이다.\n유저의 각 ...   
1        1     0  이것은 각 유저가 채팅을 입력한 맥락이 악성인지 판단하는 분류기이다.\n유저의 각 ...   
2        2     0  이것은 각 유저가 채팅을 입력한 맥락이 악성인지 판단하는 분류기이다.\n유저의 각 ...   
3        3     0  이것은 각 유저가 채팅을 입력한 맥락이 악성인지 판단하는 분류기이다.\n유저의 각 ...   
4        4     0  이것은 각 유저가 채팅을 입력한 맥락이 악성인지 판단하는 분류기이다.\n유저의 각 ...   
...    ...   ...                                                ...   
1553  1553     0  이것은 각 유저가 채팅을 입력한 맥락이 악성인지 판단하는 분류기이다.\n유저의 각 ...   
1554  1554     0  이것은 각 유저가 채팅을 입력한 맥락이 악성인지 판단하는 분류기이다.\n유저의 각 ...   
1555  1555     0  이것은 각 유저가 채팅을 입력한 맥락이 악성인지 판단하는 분류기이다.\n유저의 각 ...   
1556  1556     0  이것은 각 유저가 채팅을 입력한 맥락이 악성인지 판단하는 분류기이다.\n유저의 각 ...   
1557  1557     0  이것은 각 유저가 채팅을 입력한 맥락이 악성인지 판단하는 분류기이다.\n유저의 각 ...   

                                             Completion  
0                     중립,중립,중립,중립,중립,중립,성희롱/혐오,중립,중립,중립  
1                              

In [7]:
mywork = pd.read_csv('mydata/myworkUtf.csv', sep=',')

In [8]:
mywork.rename(columns={'Text': 'text'}, inplace=True)
mywork.rename(columns={'Completion': 'label'}, inplace=True)


In [9]:
# 'label' 컬럼을 매핑할 사전 정의
label_mapping = {
    '중립': 2,
    '욕설/비하/모욕': 0,
    '성희롱/혐오': 1
}

# 'label' 컬럼의 값 매핑
mywork['label'] = mywork['label'].map(label_mapping)

# 매핑되지 않은 행 (NaN) 제거
mywork.dropna(subset=['label'], inplace=True)
mywork.dropna(subset=['text'], inplace=True)

In [1]:

# 결과 확인
print(mywork)

NameError: name 'mywork' is not defined

In [11]:
save_with_bom_utf8(mywork, mydata+'my_pred.csv')
